In [1]:
import os
import gc
import sys
import glob
import time
import math
import numpy as np
import uproot
import pandas
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import tensorflow as tf
import keras
import keras.backend as K
from keras import regularizers
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, Dropout, AlphaDropout, Activation, BatchNormalization, Flatten, \
                                    Concatenate, PReLU, TimeDistributed, LSTM, Masking
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger
#from keras_tqdm import TQDMNotebookCallback

sys.path.insert(0, "../../python")
from common import *
from DataLoader import DataLoader, read_hdf_lock

Using TensorFlow backend.


In [2]:
class SafeModelCheckpoint(ModelCheckpoint):
    def __init__(self, filepath, **kwargs):
        super(SafeModelCheckpoint, self).__init__(filepath, **kwargs)
    
    def on_epoch_end(self, epoch, logs=None):
        read_hdf_lock.acquire()
        super(SafeModelCheckpoint, self).on_epoch_end(epoch, logs)
        read_hdf_lock.release()

In [3]:
def compile_model(model, learning_rate):
    #opt = keras.optimizers.Adam(lr=learning_rate)
    opt = keras.optimizers.Nadam(lr=learning_rate, schedule_decay=1e-4)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    metrics = [
        "accuracy", TauLosses.tau_crossentropy, TauLosses.tau_crossentropy_v2,
        TauLosses.Le, TauLosses.Lmu, TauLosses.Ljet,
        TauLosses.He, TauLosses.Hmu, TauLosses.Htau, TauLosses.Hjet,
        TauLosses.Hcat_e, TauLosses.Hcat_mu, TauLosses.Hcat_jet, TauLosses.Hbin,
        TauLosses.Hcat_eInv, TauLosses.Hcat_muInv, TauLosses.Hcat_jetInv,
        TauLosses.Fe, TauLosses.Fmu, TauLosses.Fjet, TauLosses.Fcmb
    ]
    model.compile(loss=TauLosses.tau_crossentropy_v2, optimizer=opt, metrics=metrics, weighted_metrics=metrics)

In [4]:
def close_file(f_name):
    file_objs = [ obj for obj in gc.get_objects() if ("TextIOWrapper" in str(type(obj))) and (obj.name == f_name)]
    for obj in file_objs:
        obj.close()

In [5]:
class TimeCheckpoint(Callback):
    def __init__(self, time_interval, file_name_prefix):
        self.time_interval = time_interval
        self.file_name_prefix = file_name_prefix
        self.initial_time = time.time()
        self.last_check_time = self.initial_time
    
    def on_batch_end(self, batch, logs=None):
        if self.time_interval is None or batch % 100 != 0: return
        current_time = time.time()
        delta_t = current_time - self.last_check_time
        if delta_t >= self.time_interval:
            abs_delta_t_h = (current_time - self.initial_time) / 60. / 60.
            read_hdf_lock.acquire()
            self.model.save('{}_historic_b{}_{:.1f}h.h5'.format(self.file_name_prefix, batch, abs_delta_t_h))
            read_hdf_lock.release()
            self.last_check_time = current_time
    
    def on_epoch_end(self, epoch, logs=None):
        read_hdf_lock.acquire()
        self.model.save('{}_e{}.h5'.format(self.file_name_prefix, epoch))
        read_hdf_lock.release()

In [6]:
def run_training(train_suffix, model_name, data_loader, epoch, n_epochs):

    train_name = '%s_%s' % (model_name, train_suffix)  
    log_name = "%s.log" % train_name
    if os.path.isfile(log_name):
        close_file(log_name)
        os.remove(log_name)
    csv_log = CSVLogger(log_name, append=True)

    time_checkpoint = TimeCheckpoint(12*60*60, train_name)
    callbacks = [time_checkpoint, csv_log]
    fit_hist = model.fit_generator(data_loader.generator(True), validation_data=data_loader.generator(False),
                                   steps_per_epoch=data_loader.steps_per_epoch, validation_steps=data_loader.validation_steps,
                                   callbacks=callbacks, epochs=n_epochs, initial_epoch=epoch, verbose=1)

    model.save("%s_final.hdf5" % train_name)
    return fit_hist

In [7]:
loader = DataLoader('N:/tau-ml/tuples-v2-t3/training/part_*.h5', netConf_full, 500, 10000, validation_size=6100000,
                    max_queue_size=40, n_passes=-1, return_grid=True)

print(loader.total_size, loader.data_size, loader.validation_size)

72491602 66391602 6100000


In [8]:
TauLosses.SetSFs(1, 2.5, 5, 1.5)
print(TauLosses.Le_sf, TauLosses.Lmu_sf, TauLosses.Ltau_sf, TauLosses.Ljet_sf)
model_name = "DeepTau2017v2p5"
model = LoadModel('DeepTau2017v2p5_step1_final.hdf5')
compile_model(model, 1e-3)

0.4 1.0 2.0 0.6000000000000001
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [ ]:
fit_hist = run_training('step{}'.format(2), model_name, loader, 0, 5)

Epoch 1/5
132784/132784 [==============================] - 69979s 527ms/step - loss: 0.2815 - acc: 0.8599 - tau_crossentropy: 0.1033 - tau_crossentropy_v2: 0.2777 - Le: 0.0661 - Lmu: 0.0129 - Ljet: 0.1066 - He: 0.1965 - Hmu: 0.0344 - Htau: 0.0721 - Hjet: 0.1793 - Hcat_e: 0.0310 - Hcat_mu: 0.0011 - Hcat_jet: 0.0684 - Hbin: 0.0637 - Hcat_eInv: 0.1654 - Hcat_muInv: 0.0332 - Hcat_jetInv: 0.1110 - Fe: 0.0079 - Fmu: 0.0015 - Fjet: 0.0100 - Fcmb: 0.0402 - weighted_acc: 0.8391 - weighted_tau_crossentropy: 0.1098 - weighted_tau_crossentropy_v2: 0.2815 - weighted_Le: 0.0835 - weighted_Lmu: 0.0235 - weighted_Ljet: 0.0881 - weighted_He: 0.2004 - weighted_Hmu: 0.0587 - weighted_Htau: 0.0885 - weighted_Hjet: 0.1379 - weighted_Hcat_e: 0.0320 - weighted_Hcat_mu: 0.0012 - weighted_Hcat_jet: 0.0403 - weighted_Hbin: 0.0482 - weighted_Hcat_eInv: 0.1684 - weighted_Hcat_muInv: 0.0575 - weighted_Hcat_jetInv: 0.0976 - weighted_Fe: 0.0098 - weighted_Fmu: 0.0016 - weighted_Fjet: 0.0060 - weighted_Fcmb: 0.0296 -